In [1]:
import pymonntorch as PMT
import torch

In [2]:
import conex as CNX

from conex.behaviors.neurons import (
    SimpleDendriteStructure,
    SimpleDendriteComputation,
    LIF,
    SpikeTrace,
    NeuronAxon,
    Fire,
    KWTA,
    ActivityBaseHomeostasis,
    VoltageBaseHomeostasis
)

from conex.behaviors.synapses import (
    SynapseInit,
    WeightInitializer,
    SimpleDendriticInput,
    SimpleSTDP,
    WeightNormalization,
    WeightClip,
)

In [3]:
from plot2_A import Plot
from encoding import PoissonDistribution, ResetMemory
from populationactivity import PopulationActivity
from lateral_inhibition import LateralDendriticInput
from dataGenerator import input_data_generator
import Result

In [4]:
def NeuronNetwork(input_data, input_size=0, output_size = 0, input_time = 40, rest_time = 10,    j_0=20,
    AP=0.01,
    AM=0.005,
    tau_src=3,
    tau_dst=2,
    iteration_number=3000,
    plot_scale=0,
    j_0_lateral=5,
    L4_EXC_R = 5.0,
    L4_EXC_THRESHOLD = 15,
    L4_EXC_TAU = 5.0,
    L4_EXC_V_RESET = 0,
    L4_EXC_V_REST = 5,
    output_prioritize_behaviors=[
        # KWTA(k=1),
        # ActivityBaseHomeostasis(activity_rate=3, window_size=10,updating_rate=0.2, decay_rate=0.2 ),
        
    ],
    output_behaviors={
    },
    synapse_behaviors={},
    synapse_prioritize_behaviors=[
         WeightNormalization()
    ], ):
    net = CNX.Neocortex(dt=1)

    pop1 = PMT.NeuronGroup(net=net,
                        size=input_size or input_data.shape[1],
                        behavior={**CNX.prioritize_behaviors(
                                [
                                    SimpleDendriteStructure(),
                                    SimpleDendriteComputation(),
                                    LIF(
                                        R=L4_EXC_R,
                                        threshold=L4_EXC_THRESHOLD,
                                        tau=L4_EXC_TAU,
                                        v_reset=L4_EXC_V_RESET,
                                        v_rest=L4_EXC_V_REST,
                                    ),
                                    SpikeTrace(tau_s=tau_src),
                                    Fire(),
                                    NeuronAxon(),
                                ]
                        ), 
                        **{
                        10: ResetMemory(),
                        345: PoissonDistribution(time = input_time, data=input_data, rest_time=rest_time, epsilon=0),
                        350: PopulationActivity(),
                        460: PMT.Recorder(variables=["v", "I", "T"],tag="in_recorder"),
                        461: PMT.EventRecorder("spikes", tag="in_ev_recorder"),
                        }
                        })
    
    
    pop2 = PMT.NeuronGroup(net=net,
                        size=output_size or input_data.shape[0],
                        behavior={**CNX.prioritize_behaviors(
                                [
                                    SimpleDendriteStructure(),
                                    SimpleDendriteComputation(),
                                    LIF(
                                        R=L4_EXC_R,
                                        threshold=L4_EXC_THRESHOLD,
                                        tau=L4_EXC_TAU,
                                        v_reset=L4_EXC_V_RESET,
                                        v_rest=L4_EXC_V_REST,
                                    ),
                                    SpikeTrace(tau_s=tau_dst),
                                    Fire(),
                                    NeuronAxon(),
                                    *output_prioritize_behaviors,
                                ],
                        ),
                        **{
                        350: PopulationActivity(),
                        351:Result.Result(print_details = False),
                        460: PMT.Recorder(variables=["v", "I", "T"],tag="in_recorder"),
                        461: PMT.EventRecorder("spikes", tag="in_ev_recorder"),
                        },
                        **output_behaviors,

                        },)

    syn_pop1_pop2 = PMT.SynapseGroup(
        net=net,
        src=pop1,
        dst=pop2,
        tag="Proximal",
        behavior={**CNX.prioritize_behaviors(
                    [
                        SynapseInit(),
                        WeightInitializer(mode="random"),
                        SimpleDendriticInput(current_coef=j_0),
                        SimpleSTDP(a_plus=AP, 
                                    a_minus=AM,
                                    positive_bound="soft_bound",
                                    negative_bound="soft_bound",
                                    ),
                        *synapse_prioritize_behaviors
                    ]
                ),
                **{
                    460: PMT.Recorder(variables=["weights"], tag="sg_inp_out")
                        },
                **synapse_behaviors},
    )
    
    synapse_lateral = PMT.SynapseGroup(
        net=net,
        src=pop2,
        dst=pop2,
        tag="Proximal",
        behavior=CNX.prioritize_behaviors(
                    [
                        SynapseInit(),
                        WeightInitializer(mode="random"),
                        LateralDendriticInput(current_coef=j_0_lateral),
                    ]
                    )
        )
    

    net.initialize(info=False)
    net.simulate_iterations(iteration_number )
    
    print ("---------------result----------------")
    for i in range (len(pop2.result)):
        temp = ', '.join(map(str,map(int,pop2.result[i][0]))) if pop2.result[i][0].shape[0] != pop2.size else "None"
        print (f"{i} -> {temp}")
        
    print ("-------------parameters---------------")
    
    for p in [
                "j_0={:.2f}".format(j_0),
                "iteration={:.2f}".format(iteration_number),
                "j_0_lateral={:.2f}".format(j_0_lateral),
                "input_size={:.2f}".format(input_size or input_data.shape[1]),
                "output_size={:.2f}".format(output_size or input_data.shape[0]),
                *output_prioritize_behaviors,
                *synapse_prioritize_behaviors

        ]:
        print (p)
    
    
    print ("-------------------------------------")
    Plot(
        net=net,
        ngs=[pop1, pop2],
        scaling_factor=5,
        sgs=[syn_pop1_pop2],
        label_font_size=6,
        recorder_index=460,
        env_recorder_index=461,
        )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=3,overlap=0.0,),
              iteration_number=1500,
              j_0=20,
              j_0_lateral=20,    
              AP=0.05,
              AM=0.025,)

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=4,overlap=0.0,),
              iteration_number=2000,
              j_0=20,
              j_0_lateral=20,    
              AP=0.05,
              AM=0.025,)

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=5,overlap=0.0,),
              iteration_number=2000,
              j_0=25,
              j_0_lateral=20,    
              AP=0.05,
              AM=0.025,)

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=4,overlap=0.0),
              iteration_number=2000,
              j_0=30,
              j_0_lateral=0,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=3,overlap=0.10),
              iteration_number=1500,
              j_0=30,
              j_0_lateral=10,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=3,overlap=0.0),
              iteration_number=1000,
              j_0=30,
              j_0_lateral=10,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=4,overlap=0.0),
              iteration_number=5000,
              j_0=30,
              j_0_lateral=10,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=5,overlap=0.0),
              iteration_number=1000,
              j_0=30,
              j_0_lateral=10,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=6,overlap=0.0),
              iteration_number=5000,
              j_0=30,
              j_0_lateral=10,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=7,overlap=0.0),
              iteration_number=5000,
              j_0=30,
              j_0_lateral=10,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=5,overlap=0.10),
              iteration_number=3000,
              j_0=30,
              j_0_lateral=10,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=4,overlap=0.20),
              iteration_number=3000,
              j_0=30,
              j_0_lateral=15,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=3,overlap=0.30),
              iteration_number=3000,
              j_0=30,
              j_0_lateral=15,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )

In [ ]:
NeuronNetwork(input_data=input_data_generator(size=100,output_size=3,overlap=0.50),
              iteration_number=3000,
              j_0=30,
              j_0_lateral=15,    
              AP=0.05,
              AM=0.025,
              output_prioritize_behaviors=[
                KWTA(k=1),
                ]
            )